# Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM,GRU, SimpleRNN, Dense, Dropout
import os
import tensorflow as tf
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# , confusion_matrix, ConfusionMatrixDisplay

# Functions

In [3]:
#fungsi menentukan data input dan target output
def create_my_dataset(df):
    x = []
    # y = []
    for i in range (l + 1, df.shape[0] + 1): #diulang sebanyak panjang data-l+1
        x.append(df[i-l:i,:]) #sekuen sepanjang l sebagai input
        # y.append(df_y[i,0]) #nilai output sekuen ke l+1
    x = np.array(x)
    # y = np.array(y)
    # y = y.reshape(-1,1)
    return x[50 - TARGET_DAY:]

#fungsi mengiris data training dan testing pada fitur tertentu
def get_slice(m,r,col,t): #t=0 untuk skrg, t=-1 utk sehari sebelumnya
    #get column
    df_s = df_old[col].values
    df_s = df_s.reshape(-1,1)

    #slicing
    train_s = df_s[l+t:int(r*m)+t]
    if t==0:
        test_s = df_s[int(r*m):]
    else:
        test_s = df_s[int(r*m)+t:t]

    train_s = np.array(train_s)
    train_s = train_s.reshape(-1,1)
    test_s = np.array(test_s)
    test_s = test_s.reshape(-1,1)

    return train_s,test_s

def close_target(dataframe, days, price_col):
    # dataframe = dataframe.assign(target_days='None')
    target = 'None'
    for i in range(dataframe.shape[0] - days):
        target = dataframe.loc[i + days, price_col] 
        dataframe.loc[i, f"close_{days}"] = target
    return dataframe

# Main

## Load Data

In [602]:
EMITEN = 'INCO'
TARGET_DAY = 1
file_name = f'Labelled Stock Data/Labelled_Stock_{EMITEN}.csv'
df = pd.read_csv(file_name)
df = close_target(df, TARGET_DAY, 'close')
df = df.iloc[:-50]
df

,datetime,symbol,open,high,low,close,volume,SMA_5,EMA_5,SMA_10,...,MACD_SIGNAL,UPPERBAND,MIDDLEBAND,LOWERBAND,target_1,target_5,target_10,target_20,target_50,close_1
0,2018-07-12,IDX:INCO,4223.38488,4282.45320,4134.78240,4134.78240,1.097096e+07,4054.055696,4085.980288,4034.366256,...,119.440053,4232.076359,4022.552592,3813.028825,buy,buy,hold,hold,sell,4430.12400
1,2018-07-13,IDX:INCO,4184.00600,4430.12400,4184.00600,4430.12400,1.543233e+07,4140.689232,4200.694858,4079.651968,...,122.639869,4301.886428,4034.858492,3767.830556,hold,sell,hold,hold,sell,4410.43456
2,2018-07-16,IDX:INCO,4469.50288,4577.79480,4410.43456,4410.43456,9.484069e+06,4231.260656,4270.608092,4119.030848,...,128.279305,4362.516695,4050.117808,3737.718921,sell,sell,hold,hold,sell,4282.45320
3,2018-07-17,IDX:INCO,4380.90040,4390.74512,4252.91904,4282.45320,4.593325e+06,4294.266864,4274.556462,4151.518424,...,132.817976,4382.165869,4056.516876,3730.867883,hold,hold,hold,sell,sell,4282.45320
4,2018-07-18,IDX:INCO,4282.45320,4351.36624,4262.76376,4282.45320,6.116781e+06,4308.049472,4277.188708,4179.083640,...,136.126067,4409.277329,4072.760664,3736.243999,hold,hold,hold,hold,sell,4262.76376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,2023-09-19,IDX:INCO,5832.99660,5832.99660,5636.10220,5660.71400,6.861749e+06,5749.316480,5748.868959,5791.156540,...,-125.071543,5966.146215,5830.535420,5694.924625,hold,sell,hold,hold,sell,5636.10220
1265,2023-09-20,IDX:INCO,5660.71400,5759.16120,5636.10220,5636.10220,8.668301e+06,5734.549400,5711.280039,5764.083560,...,-124.535653,5975.745040,5818.229520,5660.714000,hold,hold,sell,hold,sell,5586.87860
1266,2023-09-21,IDX:INCO,5660.71400,5685.32580,5562.26680,5586.87860,7.978185e+06,5700.092880,5669.812893,5732.088220,...,-125.287292,5987.376987,5803.462440,5619.547893,hold,hold,hold,hold,sell,5611.49040
1267,2023-09-22,IDX:INCO,5586.87860,5660.71400,5562.26680,5611.49040,5.104259e+06,5660.714000,5650.372062,5714.859960,...,-126.128304,5991.164329,5791.156540,5591.148751,hold,hold,hold,hold,sell,5586.87860


## Fit Scaler

In [603]:
df1 = df['close'].iloc[50:].values
df_y = df[f'close_{TARGET_DAY}'].iloc[50:].values

m = df1.shape[0] # panjang dataset
r = 0.8 #ratio jumlah data training terhadap dataset
l = TARGET_DAY #panjang potongan data sekuen untuk memprediksi 1 output berikutnya, UPDATE: panjang sekuens disamakan dengan target hari
n = 1

x_train, x_test, y_train, y_test = train_test_split(df1, df_y, test_size=1-r, random_state=0)
x_train = x_train.reshape([-1, 1])
y_train = y_train.reshape([-1, 1])

scaler = StandardScaler()
scaler_y = StandardScaler()
x_train = scaler.fit_transform(x_train)
y_train = scaler_y.fit_transform(y_train)
df.iloc[50:52]

,datetime,symbol,open,high,low,close,volume,SMA_5,EMA_5,SMA_10,...,MACD_SIGNAL,UPPERBAND,MIDDLEBAND,LOWERBAND,target_1,target_5,target_10,target_20,target_50,close_1
50,2018-09-25,IDX:INCO,3485.03088,3622.85696,3475.18616,3583.47808,1.251971e+07,3488.968768,3496.758044,3380.676848,...,-165.265693,3907.012399,3441.221876,2975.431353,hold,sell,sell,sell,sell,3632.70168
51,2018-09-26,IDX:INCO,3603.16752,3672.08056,3553.94392,3632.70168,7.430887e+06,3559.850752,3542.072589,3416.117840,...,-149.518217,3866.880303,3430.392684,2993.905065,hold,sell,hold,sell,sell,3642.54640


## Create Sliding Windows

In [604]:
arr = df['close'].values
arr = arr.reshape(df.shape[0], n)
df_y = df_y.reshape([-1, 1])
new_arr = create_my_dataset(arr)
print(new_arr.shape)
new_arr[0, -10:]

(1219, 1, 1)


array([[3583.47808]])

In [605]:
for i in range(new_arr.shape[0]):
    new_arr[i] = scaler.transform(new_arr[i])
df_y = scaler_y.transform(df_y)
print(new_arr.shape)
new_arr[0, -10:]

(1219, 1, 1)


array([[-0.71435621]])

In [606]:
x_train, x_test, y_train, y_test = train_test_split(new_arr, df_y, test_size=1-r, random_state=0)
print(x_train.shape[0])
print(x_test.shape[0])
x_train[0, -10:]

975
244


array([[1.91239738]])

## Modelling

In [607]:
mode = 'LSTM'
modes = ['LSTM'
         ,'GRU'
         ,'RNN'
         ]

for mode in modes:
    random.seed(0)
    tf.random.set_seed(0)
    np.random.seed(0)
    if mode == 'LSTM':
    #arsitektur LSTM
        model = Sequential()
        model.add(LSTM(units=96, return_sequences=True, input_shape=(x_train.shape[1],n))) #input n fitur
        model.add(Dropout(0.2))
        model.add(LSTM(units=96, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(units=96))
        model.add(Dropout(0.2))
        model.add(Dense(units=1)) #3

    elif mode == 'GRU':
    #arsitektur GRU
        model = Sequential()
        model.add(GRU(units=96, return_sequences=True, input_shape=(x_train.shape[1],n), activation='tanh')) #input n fitur
        model.add(Dropout(0.2))
        model.add(GRU(units=96, return_sequences=True, activation='tanh'))
        model.add(Dropout(0.2))
        model.add(GRU(units=96, activation='tanh'))
        model.add(Dropout(0.2))
        model.add(Dense(units=1)) #3

    elif mode == 'RNN':
    #arsitektur Simple RNN
        model = Sequential()
        model.add(SimpleRNN(units=96, return_sequences=True, input_shape=(x_train.shape[1],n))) #input n fitur
        model.add(Dropout(0.2))
        model.add(SimpleRNN(units=96, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(SimpleRNN(units=96))
        model.add(Dropout(0.2))
        model.add(Dense(units=1)) #3

    model.compile(loss='mean_squared_error', optimizer='adam')

    results_dir = f'Price Prediction Model/{EMITEN}'
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)

    output_name = f'{results_dir}/{mode}_{EMITEN}_Target_{TARGET_DAY}.h5'
    if (not os.path.exists(output_name)):
        model.fit(x_train, y_train, epochs=50, batch_size=32)
        model.save(output_name)

c:\Users\hufah\.conda\envs\riset_saham\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Save Prediction Results

In [608]:
modes = ['LSTM'
         ,'GRU'
         ,'RNN'
         ]

for mode in modes:
    model_name = f'{results_dir}/{mode}_{EMITEN}_Target_{TARGET_DAY}.h5'
    model = load_model(model_name)

    # make predictions
    trainPredict = model.predict(x_train)
    testPredict = model.predict(x_test)

    # invert predictions
    trainPredict = scaler_y.inverse_transform(trainPredict) #prediksi harga data training
    trainY = scaler_y.inverse_transform(y_train)
    testPredict = scaler_y.inverse_transform(testPredict) #prediksi harga data testing
    testY = scaler_y.inverse_transform(y_test)

    tmp = df.iloc[50:]
    for day in [1, 5, 10, 20, 50]:
        if TARGET_DAY != day:
            tmp = tmp.drop(f'target_{day}', axis=1)

    train_df, test_df = train_test_split(tmp, test_size=1-r,  random_state=0)

    if TARGET_DAY == 1:
        percent_threshold = 1.5
    elif TARGET_DAY == 5:
        percent_threshold = 3
    elif TARGET_DAY == 10:
        percent_threshold = 6
    elif TARGET_DAY == 20:
        percent_threshold = 9
    elif TARGET_DAY == 50:
        percent_threshold = 11
    else:
        percent_threshold = 'None'


    trainCategoryPredict = []
    for i in range(len(trainPredict)):
        pred = trainPredict[i]
        today = train_df['close'].iloc[i]
        chg = ((pred - today) / today) * 100
        labelPredict = 'None'
        labelY = 'None' 

        if pred > today and chg >= percent_threshold:
            labelPredict = 2
        elif pred < today and abs(chg) >= percent_threshold:
            labelPredict = 0
        elif abs(chg) < percent_threshold:
            labelPredict = 1

        trainCategoryPredict.append(labelPredict)

    testCategoryPredict = []
    for i in range(len(testPredict)):
        pred = testPredict[i]
        today = test_df['close'].iloc[i]
        chg = ((pred - today) / today) * 100
        labelPredict = 'None' 

        if pred > today and chg >= percent_threshold:
            labelPredict = 2
        elif pred < today and abs(chg) >= percent_threshold:
            labelPredict = 0
        elif abs(chg) < percent_threshold:
            labelPredict = 1

        testCategoryPredict.append(labelPredict)

    train_df = train_df.assign(type='train')
    train_df = train_df.assign(close_prediction=trainPredict)
    train_df = train_df.assign(prediction=trainCategoryPredict)
    test_df = test_df.assign(type='test')
    test_df = test_df.assign(close_prediction=testPredict)
    test_df = test_df.assign(prediction=testCategoryPredict)
    result_df = pd.concat([train_df, test_df])

    col = f'target_{TARGET_DAY}'
    pred_col = f'close_{TARGET_DAY}'
    le = LabelEncoder()
    le.classes_ = np.array(["sell", "hold", "buy"])
    result_df.loc[:, col] = le.transform(result_df.loc[:, col])
    result_df = result_df.rename(columns={col:'ground_truth'})
    result_df = result_df.rename(columns={pred_col:'close_ground_truth'})

    data_results_dir = f'Result Price Prediction/{EMITEN}'
    if not os.path.exists(data_results_dir):
        os.makedirs(data_results_dir)

    result_df.reset_index(drop=True, inplace=True)
    result_df.to_csv(f'{data_results_dir}/{mode}_{EMITEN}_Target_{TARGET_DAY}.csv')

31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


# Compiled Process

In [7]:
EMITEN = 'INKP'
for TARGET_DAY in [1, 5, 10, 20, 50]:
    file_name = f'Labelled Stock Data/Labelled_Stock_{EMITEN}.csv'
    df = pd.read_csv(file_name)
    df = close_target(df, TARGET_DAY, 'close')
    df = df.iloc[:-50]

    df1 = df['close'].iloc[50:].values
    df_y = df[f'close_{TARGET_DAY}'].iloc[50:].values

    m = df1.shape[0] # panjang dataset
    r = 0.8 #ratio jumlah data training terhadap dataset
    l = TARGET_DAY #panjang potongan data sekuen untuk memprediksi 1 output berikutnya, UPDATE: panjang sekuens disamakan dengan target hari
    n = 1

    x_train, x_test, y_train, y_test = train_test_split(df1, df_y, test_size=1-r, random_state=0)
    x_train = x_train.reshape([-1, 1])
    y_train = y_train.reshape([-1, 1])

    scaler = StandardScaler()
    scaler_y = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    y_train = scaler_y.fit_transform(y_train)

    arr = df['close'].values
    arr = arr.reshape(df.shape[0], n)
    df_y = df_y.reshape([-1, 1])
    new_arr = create_my_dataset(arr)
    
    for i in range(new_arr.shape[0]):
        new_arr[i] = scaler.transform(new_arr[i])
    df_y = scaler_y.transform(df_y)
    print(new_arr.shape)

    x_train, x_test, y_train, y_test = train_test_split(new_arr, df_y, test_size=1-r, random_state=0)
    print(x_train.shape[0])
    print(x_test.shape[0])

    modes = ['LSTM'
            ,'GRU'
            ,'RNN'
            ]

    for mode in modes:
        random.seed(0)
        tf.random.set_seed(0)
        np.random.seed(0)
        if mode == 'LSTM':
        #arsitektur LSTM
            model = Sequential()
            model.add(LSTM(units=96, return_sequences=True, input_shape=(x_train.shape[1],n))) #input n fitur
            model.add(Dropout(0.2))
            model.add(LSTM(units=96, return_sequences=True))
            model.add(Dropout(0.2))
            model.add(LSTM(units=96))
            model.add(Dropout(0.2))
            model.add(Dense(units=1)) #3

        elif mode == 'GRU':
        #arsitektur GRU
            model = Sequential()
            model.add(GRU(units=96, return_sequences=True, input_shape=(x_train.shape[1],n), activation='tanh')) #input n fitur
            model.add(Dropout(0.2))
            model.add(GRU(units=96, return_sequences=True, activation='tanh'))
            model.add(Dropout(0.2))
            model.add(GRU(units=96, activation='tanh'))
            model.add(Dropout(0.2))
            model.add(Dense(units=1)) #3

        elif mode == 'RNN':
        #arsitektur Simple RNN
            model = Sequential()
            model.add(SimpleRNN(units=96, return_sequences=True, input_shape=(x_train.shape[1],n))) #input n fitur
            model.add(Dropout(0.2))
            model.add(SimpleRNN(units=96, return_sequences=True))
            model.add(Dropout(0.2))
            model.add(SimpleRNN(units=96))
            model.add(Dropout(0.2))
            model.add(Dense(units=1)) #3

        model.compile(loss='mean_squared_error', optimizer='adam')

        results_dir = f'Price Prediction Model/{EMITEN}'
        if not os.path.exists(results_dir):
            os.makedirs(results_dir)

        output_name = f'{results_dir}/{mode}_{EMITEN}_Target_{TARGET_DAY}.h5'
        if (not os.path.exists(output_name)):
            model.fit(x_train, y_train, epochs=50, batch_size=32)
            model.save(output_name)

        model_name = f'{results_dir}/{mode}_{EMITEN}_Target_{TARGET_DAY}.h5'
        model = load_model(model_name)

        # make predictions
        trainPredict = model.predict(x_train)
        testPredict = model.predict(x_test)

        # invert predictions
        trainPredict = scaler_y.inverse_transform(trainPredict) #prediksi harga data training
        trainY = scaler_y.inverse_transform(y_train)
        testPredict = scaler_y.inverse_transform(testPredict) #prediksi harga data testing
        testY = scaler_y.inverse_transform(y_test)

        tmp = df.iloc[50:]
        for day in [1, 5, 10, 20, 50]:
            if TARGET_DAY != day:
                tmp = tmp.drop(f'target_{day}', axis=1)

        train_df, test_df = train_test_split(tmp, test_size=1-r,  random_state=0)

        if TARGET_DAY == 1:
            percent_threshold = 1.5
        elif TARGET_DAY == 5:
            percent_threshold = 3
        elif TARGET_DAY == 10:
            percent_threshold = 6
        elif TARGET_DAY == 20:
            percent_threshold = 9
        elif TARGET_DAY == 50:
            percent_threshold = 11
        else:
            percent_threshold = 'None'


        trainCategoryPredict = []
        for i in range(len(trainPredict)):
            pred = trainPredict[i]
            today = train_df['close'].iloc[i]
            chg = ((pred - today) / today) * 100
            labelPredict = 'None'
            labelY = 'None' 

            if pred > today and chg >= percent_threshold:
                labelPredict = 2
            elif pred < today and abs(chg) >= percent_threshold:
                labelPredict = 0
            elif abs(chg) < percent_threshold:
                labelPredict = 1

            trainCategoryPredict.append(labelPredict)

        testCategoryPredict = []
        for i in range(len(testPredict)):
            pred = testPredict[i]
            today = test_df['close'].iloc[i]
            chg = ((pred - today) / today) * 100
            labelPredict = 'None' 

            if pred > today and chg >= percent_threshold:
                labelPredict = 2
            elif pred < today and abs(chg) >= percent_threshold:
                labelPredict = 0
            elif abs(chg) < percent_threshold:
                labelPredict = 1

            testCategoryPredict.append(labelPredict)

        train_df = train_df.assign(type='train')
        train_df = train_df.assign(close_prediction=trainPredict)
        train_df = train_df.assign(prediction=trainCategoryPredict)
        test_df = test_df.assign(type='test')
        test_df = test_df.assign(close_prediction=testPredict)
        test_df = test_df.assign(prediction=testCategoryPredict)
        result_df = pd.concat([train_df, test_df])

        col = f'target_{TARGET_DAY}'
        pred_col = f'close_{TARGET_DAY}'
        le = LabelEncoder()
        le.classes_ = np.array(["sell", "hold", "buy"])
        result_df.loc[:, col] = le.transform(result_df.loc[:, col])
        result_df = result_df.rename(columns={col:'ground_truth'})
        result_df = result_df.rename(columns={pred_col:'close_ground_truth'})

        data_results_dir = f'Result Price Prediction/{EMITEN}'
        if not os.path.exists(data_results_dir):
            os.makedirs(data_results_dir)

        result_df.reset_index(drop=True, inplace=True)
        result_df.to_csv(f'{data_results_dir}/{mode}_{EMITEN}_Target_{TARGET_DAY}.csv')

(1219, 1, 1)
975
244


c:\Users\hufah\.conda\envs\riset_saham\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 23s 8ms/step - loss: 0.9999
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4092
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0652
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0417
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0362
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0378
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0355
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0319
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0356
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0343
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0336
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0335
Epoch 13/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0293
Epoch 14/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0293
Epoch 15/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0322
Epoch 16/50
31/31

31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Epoch 1/50


c:\Users\hufah\.conda\envs\riset_saham\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 19s 184ms/step - loss: 0.7852
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 202ms/step - loss: 0.8048
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 149ms/step - loss: 0.8049
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 8s 258ms/step - loss: 0.8032
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 193ms/step - loss: 0.7913
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 213ms/step - loss: 0.8000
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 237ms/step - loss: 0.7866
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 193ms/step - loss: 0.7907
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 222ms/step - loss: 0.7931
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 13s 413ms/step - loss: 0.7831
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 178ms/step - loss: 0.7935
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 201ms/step - loss: 0.7833
Epoch 13/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 194ms/step - loss: 0.7808
Epoch 14/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 206ms/step - loss: 0.7887
Epoch 15/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 177ms/step -

31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 105ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step


c:\Users\hufah\.conda\envs\riset_saham\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 32s 132ms/step - loss: 1.0333
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - loss: 1.0590
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - loss: 1.0436
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 128ms/step - loss: 1.1339
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: 0.9668
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.8337
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 1.0171
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - loss: 0.9374
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 0.9314
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 0.9038
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.9079
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 0.9281
Epoch 13/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.8589
Epoch 14/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - loss: 0.8791
Epoch 15/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.9090


31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 132ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step


# Compiled Metrics Results

In [18]:
# calculate RMSE
EMITEN = 'INKP'
modes = ['LSTM' ,'GRU','RNN']
target_days = [1, 5, 10, 20, 50]

data_results_dir = f'Result Price Prediction/{EMITEN}'
result_file_path = f'{data_results_dir}/{EMITEN}_Results.csv'
header_results = f'emiten,model,target_day,type,rmse,mape,accuracy,precision,recall,f1_score'
with open(result_file_path, 'w') as writer:
    writer.write(f'{header_results}\n')

for mode in modes:
    for day in target_days:
        result_df = pd.read_csv(f'{data_results_dir}/{mode}_{EMITEN}_Target_{day}.csv')
        train_df = result_df[result_df['type'] == 'train']
        test_df = result_df[result_df['type'] == 'test']
        trainPredict = train_df['close_prediction'].values
        testPredict = test_df['close_prediction'].values
        trainY = train_df['close_ground_truth'].values
        testY = test_df['close_ground_truth'].values
        trainCategoryPredict = train_df['prediction'].values
        testCategoryPredict = test_df['prediction'].values
        trainCatY = train_df['ground_truth'].values
        testCatY = test_df['ground_truth'].values

        rmse_train = round(math.sqrt(mean_squared_error(trainY, trainPredict)), 4)
        rmse_test = round(math.sqrt(mean_squared_error(testY, testPredict)), 4)
        mape_train = round(np.mean(np.abs((trainY - trainPredict)/trainY))*100, 4)
        mape_test = round(np.mean(np.abs((testY - testPredict)/testY))*100, 4)

        train_acc = round(accuracy_score(list(trainCatY), trainCategoryPredict), 4)
        train_prec = round(precision_score(list(trainCatY), trainCategoryPredict, average='weighted', zero_division=0), 4)
        train_rec = round(recall_score(list(trainCatY), trainCategoryPredict, average='weighted', zero_division=0), 4)
        train_f1 = round(f1_score(list(trainCatY), trainCategoryPredict, average='weighted', zero_division=0), 4)

        test_acc = round(accuracy_score(list(testCatY), testCategoryPredict), 4)
        test_prec = round(precision_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)
        test_rec = round(recall_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)
        test_f1 = round(f1_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)

        train_results = f'{EMITEN},{mode},{day},train,{rmse_train},{mape_train},{train_acc},{train_prec},{train_rec},{train_f1}'
        test_results = f'{EMITEN},{mode},{day},test,{rmse_test},{mape_test},{test_acc},{test_prec},{test_rec},{test_f1}'

        with open(result_file_path, 'a') as writer:
            writer.write('{}\n{}\n'
                         .format(train_results, test_results))

In [19]:
tmp = pd.read_csv(result_file_path)
# tmp[tmp['type'] == 'test']
lstm_acc = round(tmp[tmp['model'] == 'LSTM']['accuracy'].mean(), 4)
gru_acc = round(tmp[tmp['model'] == 'GRU']['accuracy'].mean(), 4)
rnn_acc = round(tmp[tmp['model'] == 'RNN']['accuracy'].mean(), 4)

print(f'================ {EMITEN} ==================')
print(f'LSTM Mean Accuracy: {lstm_acc}')
print(f'GRU Mean Accuracy: {gru_acc}')
print(f'RNN Mean Accuracy: {rnn_acc}')

================ INKP ==================
LSTM Mean Accuracy: 0.6144
GRU Mean Accuracy: 0.5909
RNN Mean Accuracy: 0.5192


In [20]:
# ================ ADRO ==================
# LSTM Mean Accuracy: 0.5062
# GRU Mean Accuracy: 0.4937
# RNN Mean Accuracy: 0.4632

# ================ BMRI ==================
# LSTM Mean Accuracy: 0.7172
# GRU Mean Accuracy: 0.7116
# RNN Mean Accuracy: 0.6937

# ================ INCO ==================
# LSTM Mean Accuracy: 0.5766
# GRU Mean Accuracy: 0.5622
# RNN Mean Accuracy: 0.5482